# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#CountVectorizer" data-toc-modified-id="CountVectorizer-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>CountVectorizer</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li></ul></li><li><span><a href="#TfidfVectorizer" data-toc-modified-id="TfidfVectorizer-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>TfidfVectorizer</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

### Изучение данных

Загрузим датасет и посмотрим на первые 5 строк.

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Соответственно, столбец `text` содержит текст комментария, а `toxic` — целевой признак.

Посмотрим на размер датасета.

In [3]:
data.shape

(159571, 2)

Итак, у нас присутствует около 160 тысяч комментариев, с которыми мы будем работать.

Теперь посмотрим на распределение целевого признака.

In [4]:
data.toxic.value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Видно, что только в 10% от датасета содержат токсичные комментарии. Это не очень хорошо, так как данные смещены. Примем это к сведению и используем при разделении на тренировочную и тестовую выборки.

**Вывод:**

В данном пункте были загружены исходные данные и проведено знакомство с ними.

## Обучение

Обучим две модели:
- `CountVectorizer`, которая создает векторы на основе частоты появления слов в каждой строке датасета.
- `TfidfVectorizer`, которая создает векторы на основе частоты слова в каждой строке и количестве документов, в котором встречается такое слово.

Для обоих моделей воспользуемся пайплайнами, используя функцию `Pipeline`.

Перед началом построения моделей разобьем наш датасет на тренировочную и валидационную выборки.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data.text, data.toxic, test_size=0.25,
                                                    random_state = 42, stratify=data.toxic)

Напишем функцию, которая в качестве параметров принимает пайплайн и сетку параметров, которые необходимо перебрать функцией `GridSearchCV`. Наша функция будет возвращать значение метрики F1 для тестовой выборки и матрицу неточностей.

In [6]:
def test_f1(pipeline, grid):
    clf = GridSearchCV(estimator=pipeline,
                       param_grid=grid,
                       scoring='f1',
                       cv=3) 
    clf.fit(X_train, y_train)
    best_clf = clf.best_estimator_
    print(f'Лучшие параметры модели: {clf.best_params_}')
    
    y_pred = best_clf.predict(X_test)
    print(f'Метрика F1 на тестовой выборке составила: {f1_score(y_test, y_pred)}')
    
    print('Матрица неточностей для тестовой выборки:')
    display(pd.DataFrame(confusion_matrix(y_test, y_pred),
                         columns=['true 0', 'true 1'],
                         index=['pred 0', 'pred 1']))

### CountVectorizer

Используем пайплайн с `CountVectorizer` для векторизации и `LogisticRegression` или `DecisionTreeClassifier` для классификации.

#### LogisticRegression

Создадим пайплайн с `CountVectorizer` и `LogisticRegression`.

В качестве стоп-слов для векторизации используем `ENGLISH_STOP_WORDS`.

In [18]:
pipe_c_lr = Pipeline([('vect', CountVectorizer(stop_words=ENGLISH_STOP_WORDS)),
                      ('lr', LogisticRegression(n_jobs=-1))])

grid_params_с_lr = [{'vect__max_df': [0.1, 0.2, 0.3, 0.4],
                     'lr__C': [0.1, 1.0, 10.0, 100.0]}]

test_f1(pipe_c_lr, grid_params_с_lr)

Лучшие параметры модели: {'lr__C': 10.0, 'vect__max_df': 0.2}
Метрика F1 на тестовой выборке составила: 0.7689300143359833
Матрица неточностей для тестовой выборки:


,true 0,true 1
pred 0,35170,667
pred 1,1106,2950


Видно, что модель лучше предсказывает нетоксичные комментарии, но необходимой метрики мы все же достигли, так как она составила 0.769, что выше требуемого 0.75.

Попробуем другой классификатор.

#### DecisionTreeClassifier

Повторим предыдушие шаги, перебирая другие параметры и используя `DecisionTreeClassifier` для классификации.

In [19]:
pipe_c_dt = Pipeline([('vect', CountVectorizer(stop_words=ENGLISH_STOP_WORDS)),
                      ('dt', DecisionTreeClassifier(random_state=42))])

grid_params_с_dt = [{'vect__max_df': [0.1, 0.2, 0.3, 0.4],
                     'dt__max_depth': [10, 50, 100, 200]}]

test_f1(pipe_c_dt, grid_params_с_dt)

Лучшие параметры модели: {'dt__max_depth': 100, 'vect__max_df': 0.1}
Метрика F1 на тестовой выборке составила: 0.7168563111934374
Матрица неточностей для тестовой выборки:


,true 0,true 1
pred 0,35044,793
pred 1,1347,2709


Матрица неточностей показывает, что модель предсказывает токсичные комментарии хуже, чем предыдущая.

Метрика F1 составила 0.717, что ниже требуемых 0.75.

Перейдем к другому методу векторизации.

### TfidfVectorizer

Теперь обучим модели с другим методом векторизации. Также обучим модели  другую модель. Используем пайплайн с `TfidfVectorizer` для векторизации и `LogisticRegression` или `DecisionTreeClassifier` для классификации.

#### LogisticRegression

Повторим предыдушие шаги, перебирая другие параметры и используя `LogisticRegression` для классификации.

In [21]:
pipe_t_lr = Pipeline([('vect', TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)),
                      ('lr', LogisticRegression(n_jobs=-1))])

grid_params_t_lr = [{'vect__max_df': [0.1, 0.2, 0.3, 0.4],
                     'lr__C': [0.1, 1.0, 10.0, 100.0]}]

test_f1(pipe_t_lr, grid_params_t_lr)

Лучшие параметры модели: {'lr__C': 10.0, 'vect__max_df': 0.1}
Метрика F1 на тестовой выборке составила: 0.7755889240942279
Матрица неточностей для тестовой выборки:


,true 0,true 1
pred 0,35449,388
pred 1,1241,2815


Видно, что данная модель тоже недостаточно хорошо предсказывает токсичные комментарии. 

Но метрика F1 составила 0.776, что выше необходимых 0.75 и выше чем у метода `CountVectorizer`.

Попробуем другую модель классификации.

#### DecisionTreeClassifier

Повторим предыдушие шаги, перебирая другие параметры и используя `DecisionTreeClassifier` для классификации.

In [22]:
pipe_t_dt = Pipeline([('vect', TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)),
                      ('dt', DecisionTreeClassifier(random_state=42))])

grid_params_t_dt = [{'vect__max_df': [0.1, 0.2, 0.3, 0.4],
                     'dt__max_depth': [10, 50, 100, 200]}]

test_f1(pipe_t_dt, grid_params_t_dt)

Лучшие параметры модели: {'dt__max_depth': 100, 'vect__max_df': 0.1}
Метрика F1 на тестовой выборке составила: 0.7217593224969266
Матрица неточностей для тестовой выборки:


,true 0,true 1
pred 0,35214,623
pred 1,1414,2642


Видно, что данная модель тоже недостаточно хорошо предсказывает токсичные комментарии. 

Метрика F1 составила 0.722 что ниже необходимых 0.75.

**Вывод:**

В данном пункте были обучены модели с различными методами векторизации и моделями классификации. Только модель с методом векторизации `TfidfVectorizer` и моделью классификации `LogisticRegression` показала необходимую метрику F1 более 0.75.

## Выводы

В данном проекте были построены модели, предсказывающие токсичность комментариев:
- Были загружены данные и проведено первичное знакомство с ними.
- Модель `LogisticRegression`, обученная на векторах, вычисленных методом `CountVectorizer` обладала метрикой F1 на тестовой выборке 0.769. При обучении модели на векторах, вычисленных методом `TfidfVectorizer` метрика F1 составила 0.776. Оба значения выше необходимых необходимых 0.75.
- Модель `DecisionTreeClassifier`, обученная на векторах, вычисленных методом `CountVectorizer` обладала метрикой F1 на тестовой выборке 0.717. При обучении модели на векторах, вычисленных методом `TfidfVectorizer` метрика F1 также составила 0.722.
- Таким образом, в данном проекте под требования подходит только модель, использующая векторизацию методами `CountVectorizer` или `TfidfVectorizer` и модель классификации `LogisticRegression`.